In [ ]:
import pymongo
import pandas as pd

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["animal_rescue"]
animals_collection = db["animals"]

# Read Excel file into Pandas DataFrame
df = pd.read_excel('animal_data.xlsx') 

# Convert DataFrame to dictionary
data = df.to_dict('records') 

# Insert into MongoDB collection
animals_collection.insert_many(data)

In [ ]:
from flask import Flask, render_template, request, redirect 

app = Flask(__name__)

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["animal_rescue"]
animals_collection = db["animals"]

@app.route("/")
def index():
    animals = animals_collection.find()
    return render_template("index.html", animals=animals)

@app.route("/animal/<animal_id>")
def animal_profile(animal_id):
    animal = animals_collection.find_one({"_id": animal_id})
    return render_template("profile.html", animal=animal) 

@app.route("/search")
def search():
    name = request.args.get("name") 
    query = {"name": {"$regex": name, "$options": "i"}}
    animals = animals_collection.find(query)
    return render_template("search.html", animals=animals)
    
if __name__ == "__main__":
    app.run(debug=True)